In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [2]:
mno_asiapacific = "https://en.wikipedia.org/wiki/List_of_mobile_network_operators_of_the_Asia_Pacific_region"
mno_europe = "https://en.wikipedia.org/wiki/List_of_mobile_network_operators_of_Europe"
mno_americas = "https://en.wikipedia.org/wiki/List_of_mobile_network_operators_of_the_Americas"
mno_africas = "https://en.wikipedia.org/wiki/List_of_mobile_network_operators_of_the_Middle_East_and_Africa"

mno_urls = [mno_asiapacific, mno_europe, mno_americas, mno_africas]

```sql
with testData1 as (
select * from unnest([
      struct
      (1799867122 as user_id, 158 as product_id, timestamp (null) as expire_time_after_purchase,  70000000 as transaction_id, timestamp '2020-11-23 09:01:00' as created_at),
      (1799867122,158,timestamp (null),70000001,timestamp '2020-11-23 09:15:00.042308 UTC'),
      (1799867122,158,timestamp (null),70000002,timestamp '2020-11-23 09:30:00.042308 UTC'),
      (1799867122,158,timestamp (null),70000003,timestamp '2020-11-23 09:45:00.042308 UTC')
      ]
  ) as t
)

Select * from testData1
```

In [3]:
def parse_wiki(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text)
    df_dict = {
        "CountryName": [],
        "MNOName": [],
        "Rank": [],
        "Virtual": []
    }

    overalldiv = soup.find("div", {"class": "mw-parser-output"})

    countryHeads = overalldiv.findAll("h2")
    isVirtual = False
    for countryHead in countryHeads:
        isVirtual = False
        countryName = countryHead.find("span").text
        if(countryName.strip() == ""):
            continue
        #print("Country Name", countryName)
        b = countryHead.find_next("table")
        if(b is None):
            continue
        rows = b.find_all("tr")[1:]
        for row in rows:
            cols = row.find_all("td")
            if(len(cols) != 5 and len(cols) != 6):
                if(len(row.find_all("th")) == 1 and (row.find("a") is not None) and re.search("virtual", row.find("a").text, re.IGNORECASE)):
                    isVirtual = True
                continue
            rank = cols[0].text
            operator = cols[1].text
            #print("Rank", rank)
            #print("Operator", operator)

            df_dict["CountryName"].append(countryName)
            df_dict["Rank"].append(rank)
            df_dict["MNOName"].append(operator)
            df_dict["Virtual"].append(isVirtual)

    df = pd.DataFrame(df_dict)
    return df

In [4]:
import geohash
# Retrieving Datasets as dataframes
ndt7_df   = pd.read_csv("./data/mlab_starlink_data/ndt7_cities_overview.csv").set_index("ClientGeohash")
ndt7_df = ndt7_df.sort_values(by="MeasurementCount", ascending=False)
ndt7_df["lat"] = [geohash.decode(gh)[0] for gh in ndt7_df.index]
ndt7_df["lon"] = [geohash.decode(gh)[1] for gh in ndt7_df.index]

In [5]:
df = pd.concat([parse_wiki(url) for url in mno_urls])

In [7]:
df.to_csv("data/mno_list.txt")

In [8]:
#df = df[(df["Rank"] == "1") & (df["Virtual"] == False)]
df_filtered = df[df["CountryName"].isin(ndt7_df["ClientCountry"].drop_duplicates().values)]
df_filtered.to_csv("data/mno_list_starlinkfiltered.csv")

In [12]:
sorted(set(df["CountryName"]))

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Andorra',
 'Angola',
 'Anguilla',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bermuda',
 'Bhutan',
 'Bolivia',
 'Bonaire',
 'Bosnia and Herzegovina',
 'Bougainville',
 'Brazil',
 'British Virgin Islands',
 'Brunei Darussalam',
 'Burkina Faso',
 'Burundi',
 'Cabo Verde',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cayman Islands',
 'Central African Republic',
 'Chad',
 'Chile',
 'Colombia',
 'Comoros',
 'Congo',
 'Cook Islands',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Democratic Republic of the Congo',
 'Denmark',
 'Djibouti',
 'Dominica',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Equatorial Guinea',
 'Eritrea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Falkland Islands',
 'Faroe Islands',
 'Fiji',
 'Finland',
 'France',
 'French Polynesia

In [13]:
df

,CountryName,MNOName,Rank,Virtual
0,Afghanistan,MTN,1,False
1,Afghanistan,Roshan,2,False
2,Afghanistan,Etisalat Afghanistan,3,False
3,Afghanistan,Afghan Wireless,4,False
4,Afghanistan,Salaam,5,False
...,...,...,...,...
219,Zambia,Airtel,2,False
220,Zambia,Zamtel,3,False
221,Zimbabwe,Econet,1,False
222,Zimbabwe,telecel,2,False


In [14]:
sql_template = """
with mnoData as (
select * from unnest([struct {}]) as t
)
"""

In [15]:
first_row = True
def return_rowstr(a):
    global first_row
    if first_row:
        first_row = False
        return "({})".format(
            ", ".join([
                "'{}' as CountryName".format(str(a["CountryName"]).strip()),
                "'{}' as MNOName".format(str(a["MNOName"]).strip()),
                "'{}' as Rank".format(str(a["Rank"]).strip()),
                "{} as Virtual".format(str(a["Virtual"]).strip())])
        )
    else:
        return str((str(a["CountryName"]).strip(), str(a["MNOName"]).strip(), str(a["Rank"]).strip(), eval(str(a["Virtual"]).strip())))

sql_str = sql_template.format(",\n".join([return_rowstr(a) for a in df.iloc]))

In [16]:
with open("mno_sql_query.txt", "w+") as file:
    file.write(sql_str)

## AI Powered list

In [33]:
mno_df = pd.read_csv("data/mno_list_withasn_aipowered.csv")
mno_df = mno_df.dropna()
mno_df["IslandNation"] = mno_df["IslandNation"] == "y"
mno_df["ManualCheck"] = mno_df["ManualCheck"] == "y"
mno_df

,CountryName,MonileOperator,ASN,ManualCheck,IslandNation
0,Netherlands,KPN,AS1136,True,False
1,Netherlands,T-Mobile,AS50266,True,False
2,Netherlands,Tele2,AS13127,True,False
3,Dominican Republic,Claro,AS6400,True,True
4,Dominican Republic,Altice Dominicana,AS28118,True,True
...,...,...,...,...,...
103,Saint Barthélemy,Orange Caraïbe,???,False,True
104,Saint Barthélemy,Digicel,???,False,True
105,Sweden,Telia Company,AS3301,False,False
106,Sweden,Tele2 Sweden,AS1257,False,False


In [34]:
old_mno_df = pd.read_csv("data/mno_list_withasn_aipowered_OLD.csv")

In [37]:
len(set(mno_df["CountryName"]))

34

In [36]:
set(mno_df["CountryName"]).difference(set(old_mno_df["CountryName"]))

{'Czechia', 'Greece', 'Guadeloupe', 'Kenya', 'Mozambique', 'Poland', 'Sweden'}

In [38]:
set(old_mno_df["CountryName"]).difference(set(mno_df["CountryName"]))

{nan}

In [28]:
# TODO: build a query that which generates where clauses for country2citydiagnostic.csv
where_template = "(\n\t{}\n)".format("OR\n\t".join([ "(client.Geo.CountryName = \"{}\" AND client.Network.ASNumber = {})".format(row["CountryName"], int(row["ASN"][2:])) for _, row in mno_df.iterrows() if "???" not in row["ASN"]]))

In [29]:
print(where_template)

(
	(client.Geo.CountryName = "Netherlands" AND client.Network.ASNumber = 1136)OR
	(client.Geo.CountryName = "Netherlands" AND client.Network.ASNumber = 50266)OR
	(client.Geo.CountryName = "Netherlands" AND client.Network.ASNumber = 13127)OR
	(client.Geo.CountryName = "Dominican Republic" AND client.Network.ASNumber = 6400)OR
	(client.Geo.CountryName = "Dominican Republic" AND client.Network.ASNumber = 28118)OR
	(client.Geo.CountryName = "Dominican Republic" AND client.Network.ASNumber = 27887)OR
	(client.Geo.CountryName = "Canada" AND client.Network.ASNumber = 577)OR
	(client.Geo.CountryName = "Canada" AND client.Network.ASNumber = 812)OR
	(client.Geo.CountryName = "Canada" AND client.Network.ASNumber = 852)OR
	(client.Geo.CountryName = "Nigeria" AND client.Network.ASNumber = 29465)OR
	(client.Geo.CountryName = "Nigeria" AND client.Network.ASNumber = 36873)OR
	(client.Geo.CountryName = "Nigeria" AND client.Network.ASNumber = 328309)OR
	(client.Geo.CountryName = "Austria" AND client.Net

## MNO Ranking

In [148]:
import re
import argparse
import sys
import json
import requests

In [149]:
URL = "https://api.asrank.caida.org/v2/graphql"
decoder = json.JSONDecoder()
encoder = json.JSONEncoder()

######################################################################
## Main code
######################################################################
def query_asn(asn):
    query = AsnQuery(asn)
    request = requests.post(URL,json={'query':query})
    if request.status_code == 200:
        #print (request.json())
        return request.json()
    else:
        print ("Query failed to run returned code of %d " % (request.status_code))
        return None

######################################################################
## Queries
######################################################################

def AsnQuery(asn): 
    return """{
        asn(asn:"%i") {
            asn
            asnName
            rank
            country {
                iso
                name
            }
            asnDegree {
                provider
                peer
                customer
                total
                transit
                sibling
            }
            announcing {
                numberPrefixes
                numberAddresses
            }
        }
    }""" % (int(asn[2:]))
#run the main method

results = []
for _, row in mno_df.iterrows():
    results.append(query_asn(row["ASN"])["data"]["asn"])

In [150]:
caida_df = pd.DataFrame(results)
caida_df = caida_df.set_index("asn")

In [151]:
def getrank(asn):
    return caida_df.loc[[asn[2:]]].iloc[0]["rank"]
getrank("AS50266")

1284

In [152]:
mno_df["asrank"] = mno_df["ASN"].apply(getrank)
#mno_df = mno_df.set_index(["CountryName", "ASN"])

In [157]:
mno_df

,CountryName,MonileOperator,ASN,ManualCheck,IslandNation,asrank
0,Netherlands,KPN,AS1136,True,False,777
1,Netherlands,T-Mobile,AS50266,True,False,1284
2,Netherlands,Tele2,AS13127,True,False,1378
3,Dominican Republic,Claro,AS6400,True,True,2796
4,Dominican Republic,Altice Dominicana,AS28118,True,True,3070
...,...,...,...,...,...,...
76,Spain,Orange,AS12479,True,False,327
77,Spain,Vodafone,AS12430,True,False,348
78,Japan,NTT Docomo,AS9605,True,False,11586
79,Japan,KDDI,AS2516,True,False,91


In [158]:
sss = "\n".join([
    "{} & {} & {} & {}\\\\ \\hline".format(
            row["CountryName"],
            row["MonileOperator"],
            row["ASN"], row["asrank"]) for _, row in mno_df.sort_values(["CountryName", "asrank"]).iterrows()]
)

print(sss)

Australia & Telstra & AS1221 & 52\\ \hline
Australia & Optus & AS4804 & 5300\\ \hline
Australia & Vodafone & AS133612 & 7072\\ \hline
Austria & A1 Telekom Austria & AS8447 & 151\\ \hline
Austria & Magenta Telekom & AS25255 & 494\\ \hline
Austria & T-Mobile Austria & AS8412 & 502\\ \hline
Belgium & Proximus & AS5432 & 957\\ \hline
Belgium & Telenet & AS6848 & 1202\\ \hline
Belgium & Orange & AS47377 & 3083\\ \hline
Brazil & TIM & AS26615 & 72\\ \hline
Brazil & Vivo & AS18881 & 819\\ \hline
Brazil & Claro & AS28573 & 7011\\ \hline
Canada & Bell & AS577 & 84\\ \hline
Canada & Telus & AS852 & 203\\ \hline
Canada & Rogers & AS812 & 244\\ \hline
Chile & Movistar & AS7418 & 3848\\ \hline
Chile & Claro & AS27995 & 7102\\ \hline
Chile & Entel & AS27651 & 11671\\ \hline
Colombia & Tigo Colombia/EPM Telecomunicaciones & AS13489 & 1323\\ \hline
Colombia & Claro & AS10620 & 11592\\ \hline
Colombia & Colombia Móvil & AS27831 & 11603\\ \hline
Dominican Republic & Claro & AS6400 & 2796\\ \hline
Domini